In [1]:
import os
import numpy as np
import librosa
from sklearn.preprocessing import StandardScaler
import pickle
from sklearn.decomposition import PCA
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances
from queue import PriorityQueue


In [2]:
current_directory = os.getcwd()


In [3]:
import pickle

# Cargar la lista desde el archivo .pkl
with open('canciones.pkl', 'rb') as file:
    canciones_cargadas = pickle.load(file)

# Ahora canciones_cargadas contiene la lista que guardaste anteriormente
#print(len(canciones_cargadas))
#print(canciones_cargadas[0])


In [4]:
import numpy as np

# Cargar el archivo features.pkl con allow_pickle=True
#caractersiticas_Existentes = np.load('features.pkl', allow_pickle=True)
with open('id_caracteristicas.txt', 'r', encoding='utf-8') as file:
    nombres_id = file.readlines()

#print(len(nombres_id))

In [5]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
#NORMALIZAR data, para mejor busquedas 
data_normalized = scaler.fit_transform(canciones_cargadas)



In [6]:
import os
import numpy as np
import librosa
from sklearn.preprocessing import StandardScaler


In [7]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances
from queue import PriorityQueue

def knn_search_priority_queue(query, k):
    # Hasta el de mejor resultados
    similarities = cosine_similarity(query.reshape(1, -1), data_normalized).flatten()
    
    # cola de prioridad
    priority_queue = PriorityQueue()
    for i, sim in enumerate(similarities):
        priority_queue.put((-sim, nombres_id[i]))  # por ser una cola para sacar los mejores(mas abajo ) desde abajo los metemos
    

    neighbors = []
    for _ in range(k):
        sim, neighbor = priority_queue.get()
        neighbors.append((neighbor, -sim))
    
    return neighbors


In [8]:

k = 5
nueva_cancion_features =  data_normalized[1631]#pasarle una cancion normalizada que feo yo queria pasarle una nueva


In [9]:
#Ya no es neceesario nomrlaizar
"""
query_normalized = scaler.transform(nueva_cancion_features.reshape(1, -1))
knn_result_priority_queue = knn_search_priority_queue((query_normalized), k)
"""
knn_result_priority_queue = knn_search_priority_queue((nueva_cancion_features), k)
print("Vecinos más cercanos (KNN con cola de prioridad): ")

for i, data in knn_result_priority_queue:
    print(i,":" ,data)


Vecinos más cercanos (KNN con cola de prioridad): 
0RcZp172ZchsrpRWgBKkyt.mp3
 : 1.0000000000000007
1acz4dYqwZTZw6TgRBmmf2.mp3
 : 0.9028725695470132
26NX1wPt1TRCH536yocd6i.mp3
 : 0.8900716896348277
0Ow2s49u3akTmiLDmb1GU0.mp3
 : 0.8804939022845187
0nDI66E3QkCqeykXMMBo7w.mp3
 : 0.8786401404586751


In [10]:
import numpy as np

def knn_range_search(query, radio):

    vecinos_en_radio = []

    for i, sample in enumerate(data_normalized):
        distancia = euclidean_distances(query.reshape(1, -1), sample.reshape(1, -1))[0][0]
        if distancia is not None and distancia <= radio:
            vecinos_en_radio.append((nombres_id[i], distancia))
    
    vecinos_en_radio = sorted(vecinos_en_radio, key=lambda x: x[1])

    return vecinos_en_radio


In [11]:
import time

In [12]:
r=50
rango= knn_range_search((nueva_cancion_features), r)
print(f"Vecinos más cercanos (range search):")
for i, data in rango[:k]:
    print(i, data)



Vecinos más cercanos (range search):
0RcZp172ZchsrpRWgBKkyt.mp3
 0.0
1acz4dYqwZTZw6TgRBmmf2.mp3
 10.76116264900339
26NX1wPt1TRCH536yocd6i.mp3
 12.106064491649589
1Jgaw4deeHcbldSmKbP5h3.mp3
 12.183344420725206
0nDI66E3QkCqeykXMMBo7w.mp3
 12.334539709835617


In [13]:
def knn_sequential(query, k):
    distances = []
    for sample in data_normalized:
        distance = euclidean_distances(query.reshape(1, -1), sample.reshape(1, -1))[0][0]
        distances.append(distance)
    
    indices = np.argsort(distances)[:k]#indices de manera ascendendete
    
    neighbors = []
    for index in indices:
        neighbors.append((nombres_id[index], distances[index]))
    
    return neighbors


In [14]:
k=5

start_time = time.time()
knn_result_knn_sequential= knn_sequential((nueva_cancion_features), k)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Tiempo de ejecución: {elapsed_time} segundos")
print(f"Vecinos más cercanos (KNN Secuencial):")
for i, data in knn_result_knn_sequential:
    print(i, data)


Tiempo de ejecución: 0.9537875652313232 segundos
Vecinos más cercanos (KNN Secuencial):
0RcZp172ZchsrpRWgBKkyt.mp3
 0.0
1acz4dYqwZTZw6TgRBmmf2.mp3
 10.76116264900339
26NX1wPt1TRCH536yocd6i.mp3
 12.106064491649589
1Jgaw4deeHcbldSmKbP5h3.mp3
 12.183344420725206
0nDI66E3QkCqeykXMMBo7w.mp3
 12.334539709835617


In [15]:
import pandas as pd
import numpy as np
import  faiss

In [17]:
def train_index_HNSWFlat(data):
    # Inicializa el índice de FAISS con HNSW (Hierarchical Navigable Small World)
    M = 32  # Número de vecinos en la lista de entrada
    efConstruction = 100  # explorar al construir canciones mientras mayor es mejor la precision y mas demora
    dimension = data_normalized.shape[1]   # Dimensión de tus características
    quantizer = faiss.IndexHNSWFlat(dimension, M, faiss.METRIC_L2)#distancia eulediana
    quantizer.hnsw.efConstruction = efConstruction
    index = quantizer
    index.add(data)
    return index
"""
Sí, eso es correcto. En el contexto del índice HNSW con el parámetro M establecido en 32, cada punto (en este caso, cada canción) 
está directamente conectado a un máximo de 32 puntos en su lista de entrada. Estos son sus "vecinos cercanos" en el espacio de características.
""""""En otras palabras, cada canción en tu conjunto de datos tiene conexiones directas con hasta M 
canciones más cercanas según la métrica de distancia especificada (en este caso, la distancia L2)."""
def knn_faiss_HNSWFlat(indice, query_object, k):
    distances, indices = indice.search(np.expand_dims(query_object, axis=0), k)
    resultados = []
    for i in range(k):
        distancia = distances[0][i]
        etiqueta = nombres_id[indices[0][i]]
        resultados.append((distancia, etiqueta))
    return resultados

# Entrenamiento con IndexHNSWFlat
index_train = train_index_HNSWFlat(data_normalized)



In [18]:


query_example = nueva_cancion_features
start_time = time.time()
k_result = knn_faiss_HNSWFlat(index_train, query_example, 5)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Tiempo de ejecución: {elapsed_time} segundos")

print(len(k_result))

print("Resultados de búsqueda KNN con FAISS (IndexHNSWFlat):")
for a, i in k_result:
    print(f"Distancia: {a}, Etiqueta: {i}")

Tiempo de ejecución: 0.0020024776458740234 segundos
5
Resultados de búsqueda KNN con FAISS (IndexHNSWFlat):
Distancia: 0.0, Etiqueta: 0RcZp172ZchsrpRWgBKkyt.mp3

Distancia: 115.80262756347656, Etiqueta: 1acz4dYqwZTZw6TgRBmmf2.mp3

Distancia: 146.55679321289062, Etiqueta: 26NX1wPt1TRCH536yocd6i.mp3

Distancia: 148.43386840820312, Etiqueta: 1Jgaw4deeHcbldSmKbP5h3.mp3

Distancia: 152.140869140625, Etiqueta: 0nDI66E3QkCqeykXMMBo7w.mp3



In [20]:
matrices_array = np.array(data_normalized)

# Número de dimensiones deseadas después de PCA
n_dimensions = 100

# Crear un objeto PCA
pca = PCA(n_components=n_dimensions)

# Aplicar PCA a las matrices
matrices_reduced = pca.fit_transform(matrices_array)

# Imprimir el tipo y los datos del array resultante después de PCA
"""print(type(matrices_reduced))
print(matrices_reduced)
print(len(matrices_reduced[0]))"""

from rtree import index
p = index.Property()
p.dimension = 100
p.buffering_capacity = 8
idx = index.Index(properties=p)

for i, sample in enumerate(matrices_reduced):
    idx.insert(i, sample)
    #print("insertando",i)



In [21]:
nearest_neighbors_indices = list(idx.nearest((matrices_reduced[3481]), 5)) # 5 vecinos más cercanos a la canción 3481

# Print the names of the nearest songs
for i in nearest_neighbors_indices:
    print(nombres_id[i])

1KyA5lXW7cAgMLwxRdVAu3.mp3

0IXpUl1fn2QZcBavfuq0H4.mp3

0rZTJmLQx7IquBctp98d0z.mp3

0fsj3HxzBHev6EJ7OLFE3r.mp3

0TcOuAT1fL2RA1YUgTrBvK.mp3

